# Data Loading

To get started we load text frim H. G. Wells' [Time Machine](http://www.gutenberg.org/ebooks/35). This is fairly small corpus of just over 30,000 words but for the purpose of what we want to illustrate this is just fine. More realistic document collection contain many billions of words. The following function read the dataset into a list of sentences, each sentence is a string. Here we ignore punctuation and capitalization.

In [3]:
import collections
import re

def read_time_machine():
    """Load the time machine book into a list of sentences."""
    with open('./timemachine.txt', 'r') as f:
        lines = f.readlines()
    clean_text =[re.sub('[^A-Za-z]+', ' ', line.strip().lower()) 
                 for line in lines]
    return clean_text

lines = read_time_machine()
'# sentences %d' % len(lines)

'# sentences 3221'

In [6]:
lines[8]

'the time traveller for so it will be convenient to speak of him '

# Tokenization

For each sentence, we split it into a list of tokens. A token is a data point the model will train and predict. The following function support split a sentence into words or characters, and return a list of split sentences.

In [8]:
def tokenize(lines, token='word'):
    """Split sentences into word or char tokens"""
    if token == 'word':
        return [line.split(' ') for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('ERROR: unkown token type '+token)

tokens = tokenize(lines)
tokens[0:2]

[['the', 'time', 'machine', 'by', 'h', 'g', 'wells', ''], ['']]

# Vocabulary

The string type of the token is inconvenient to be used by models, which take numerical inputs. Now let's build a dictionary, often called vocabulary as well, to map string tokens into numerical indices starting from 0. To do so, we first count the unique tokens in all documents, called corpus, and then assign a numerical index to each unique token according to its frequency. Rarely appeared tokens are often removed to reduce the compexity. A token doesn't exist in corpus or has been removed is mapped into a special unknown ("unk") token. We optionally add another special tokens: "pad" a token for padding, "bos" to represent the beginning for a sentence, and "eos" for the ending of a sentence.

In [13]:
def count_corpus(sentences):
    # Flatten a list of token lists into a list of tokens
    tokens = [tk for line in sentences for tk in line]
    return collections.Counter(tokens)

counter = count_corpus(tokens)

In [12]:
lines[8]

'the time traveller for so it will be convenient to speak of him '

In [14]:
counter

Counter({'the': 2261,
         'time': 200,
         'machine': 85,
         'by': 103,
         'h': 1,
         'g': 1,
         'wells': 9,
         '': 1282,
         'i': 1267,
         'traveller': 61,
         'for': 221,
         'so': 112,
         'it': 437,
         'will': 37,
         'be': 93,
         'convenient': 5,
         'to': 695,
         'speak': 6,
         'of': 1155,
         'him': 40,
         'was': 552,
         'expounding': 2,
         'a': 816,
         'recondite': 1,
         'matter': 6,
         'us': 35,
         'his': 129,
         'grey': 11,
         'eyes': 35,
         'shone': 8,
         'and': 1245,
         'twinkled': 1,
         'usually': 3,
         'pale': 10,
         'face': 38,
         'flushed': 2,
         'animated': 3,
         'fire': 30,
         'burned': 6,
         'brightly': 4,
         'soft': 16,
         'radiance': 1,
         'incandescent': 1,
         'lights': 1,
         'in': 541,
         'lilies': 1,
     

In [15]:
counter.items()

dict_items([('the', 2261), ('time', 200), ('machine', 85), ('by', 103), ('h', 1), ('g', 1), ('wells', 9), ('', 1282), ('i', 1267), ('traveller', 61), ('for', 221), ('so', 112), ('it', 437), ('will', 37), ('be', 93), ('convenient', 5), ('to', 695), ('speak', 6), ('of', 1155), ('him', 40), ('was', 552), ('expounding', 2), ('a', 816), ('recondite', 1), ('matter', 6), ('us', 35), ('his', 129), ('grey', 11), ('eyes', 35), ('shone', 8), ('and', 1245), ('twinkled', 1), ('usually', 3), ('pale', 10), ('face', 38), ('flushed', 2), ('animated', 3), ('fire', 30), ('burned', 6), ('brightly', 4), ('soft', 16), ('radiance', 1), ('incandescent', 1), ('lights', 1), ('in', 541), ('lilies', 1), ('silver', 6), ('caught', 10), ('bubbles', 1), ('that', 443), ('flashed', 4), ('passed', 13), ('our', 57), ('glasses', 1), ('chairs', 2), ('being', 14), ('patents', 1), ('embraced', 1), ('caressed', 2), ('rather', 18), ('than', 34), ('submitted', 1), ('sat', 22), ('upon', 113), ('there', 127), ('luxurious', 1), ('

In [16]:
token_freqs = sorted(counter.items(),
                                 key=lambda x: x[1],
                                 reverse=True)
token_freqs

[('the', 2261),
 ('', 1282),
 ('i', 1267),
 ('and', 1245),
 ('of', 1155),
 ('a', 816),
 ('to', 695),
 ('was', 552),
 ('in', 541),
 ('that', 443),
 ('my', 440),
 ('it', 437),
 ('had', 354),
 ('me', 281),
 ('as', 270),
 ('at', 243),
 ('for', 221),
 ('with', 216),
 ('but', 204),
 ('time', 200),
 ('were', 158),
 ('this', 152),
 ('you', 137),
 ('on', 137),
 ('then', 134),
 ('his', 129),
 ('there', 127),
 ('he', 123),
 ('have', 122),
 ('they', 122),
 ('from', 122),
 ('one', 120),
 ('all', 118),
 ('not', 114),
 ('into', 114),
 ('upon', 113),
 ('little', 113),
 ('so', 112),
 ('is', 106),
 ('came', 105),
 ('by', 103),
 ('some', 94),
 ('be', 93),
 ('no', 92),
 ('could', 92),
 ('their', 91),
 ('said', 89),
 ('saw', 88),
 ('down', 87),
 ('them', 86),
 ('machine', 85),
 ('which', 85),
 ('very', 85),
 ('or', 84),
 ('an', 84),
 ('we', 82),
 ('now', 79),
 ('what', 77),
 ('been', 75),
 ('these', 74),
 ('like', 74),
 ('her', 74),
 ('out', 73),
 ('seemed', 72),
 ('up', 71),
 ('man', 70),
 ('about', 70),


In [17]:
token_freqs.sort(key=lambda x: x[1],
                reverse=True)
token_freqs

[('the', 2261),
 ('', 1282),
 ('i', 1267),
 ('and', 1245),
 ('of', 1155),
 ('a', 816),
 ('to', 695),
 ('was', 552),
 ('in', 541),
 ('that', 443),
 ('my', 440),
 ('it', 437),
 ('had', 354),
 ('me', 281),
 ('as', 270),
 ('at', 243),
 ('for', 221),
 ('with', 216),
 ('but', 204),
 ('time', 200),
 ('were', 158),
 ('this', 152),
 ('you', 137),
 ('on', 137),
 ('then', 134),
 ('his', 129),
 ('there', 127),
 ('he', 123),
 ('have', 122),
 ('they', 122),
 ('from', 122),
 ('one', 120),
 ('all', 118),
 ('not', 114),
 ('into', 114),
 ('upon', 113),
 ('little', 113),
 ('so', 112),
 ('is', 106),
 ('came', 105),
 ('by', 103),
 ('some', 94),
 ('be', 93),
 ('no', 92),
 ('could', 92),
 ('their', 91),
 ('said', 89),
 ('saw', 88),
 ('down', 87),
 ('them', 86),
 ('machine', 85),
 ('which', 85),
 ('very', 85),
 ('or', 84),
 ('an', 84),
 ('we', 82),
 ('now', 79),
 ('what', 77),
 ('been', 75),
 ('these', 74),
 ('like', 74),
 ('her', 74),
 ('out', 73),
 ('seemed', 72),
 ('up', 71),
 ('man', 70),
 ('about', 70),


In [20]:
class Vocab(object):
    def __init__(self, tokens, min_freq=0, use_special_tokens=False):
        # Sort according to frequencies
        counter = count_corpus(tokens)
        self.token_freqs = sorted(counter.items(),
                                 key=lambda x: x[1],
                                 reverse=True)
        if use_special_tokens:
            # padding, begin of sentence, end of sentence, unknown
            self.pad, self.bos, self.eos, self.unk = (0, 1, 2, 3)
            uniq_tokens = ['<pad>', '<bos>', '<eos>', '<unk>']
        else:
            self.unk, uniq_tokens = 0, ['<unk>']
        uniq_tokens += [token for token, freq in self.token_freqs
                       if freq>= min_freq and 
                        token not in uniq_tokens]
        self.idx_to_token, self.token_to_idx = [], dict()
        for token in uniq_tokens:
            self.idx_to_token.append(token)
            self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[idx] for idx in indices]
    
def count_corpus(sentences):
    # Flatten a list of token lists into a list of tokens
    tokens = [tk for line in sentences for tk in line]
    return collections.Counter(tokens)

We construct a vocabulary with the time machine dataset as the corpus, and then print the map between a few tokens to indices.

In [21]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('', 2), ('i', 3), ('and', 4), ('of', 5), ('a', 6), ('to', 7), ('was', 8), ('in', 9)]


After that, we can convert each sentence into a list of numerical indices. To illustrate things we print two sentences with their corresponding indices.

In [22]:
for i in range(8, 10):
    print('words:', tokens[i])
    print('indices:', vocab[tokens[i]])

words: ['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him', '']
indices: [1, 20, 72, 17, 38, 12, 116, 43, 681, 7, 587, 5, 109, 2]
words: ['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
indices: [8, 1421, 6, 2186, 588, 7, 127, 26, 331, 128, 440, 4]


# Put All Things Together

We packaged the above code in the ``load_corpus_time_machine`` function, which returns ``corpus``, a list of token indices, and ``vocab``, the vocabulary. The modification we did here is that ``corpus`` is a single list, not a list of token lists, since we do not use the sequence information in the following models. Besides, we use character tokens to simplify the training in later sections.

In [26]:
def load_corpus_time_machine(max_tokens=-1):
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    corpus = [vocab[tk] for line in tokens for tk in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(171489, 28)

In [27]:
vocab.token_to_idx.items()

dict_items([('<unk>', 0), (' ', 1), ('e', 2), ('t', 3), ('a', 4), ('i', 5), ('n', 6), ('o', 7), ('s', 8), ('h', 9), ('r', 10), ('d', 11), ('l', 12), ('m', 13), ('u', 14), ('c', 15), ('f', 16), ('w', 17), ('g', 18), ('y', 19), ('p', 20), ('b', 21), ('v', 22), ('k', 23), ('x', 24), ('z', 25), ('j', 26), ('q', 27)])